In [1]:
!pip install langgraph langchain langchain_openai -qqq

In [4]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [5]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph
from typing import TypedDict, Annotated, Sequence, Dict, Optional
import operator

# Define your state
class AgentState(TypedDict):
    messages: Annotated[Sequence[str], operator.add]
    summary: str  # or any other relevant state information
    extracted_info: Optional[Dict[str, str]]  # Dictionary to store extracted information for each keyword

# Initialize the ChatOpenAI model
model = ChatOpenAI()

# Define your nodes (functions)
def process_message(message):
    prompt = f"Summarize the following message: {message}"
    response = model.invoke(prompt)
    if hasattr(response, 'content'):
        return response.content.strip()
    print("Unexpected response structure:", response)
    if isinstance(response, dict):
        try:
            processed_text = response['choices'][0]['text'].strip()
            print("Processed text using dict-like access:", processed_text)
            return processed_text
        except (KeyError, TypeError, IndexError):
            print("Error accessing response using dict-like structure")

    # Return an error message if the correct format cannot be determined
    return "Error processing message"

def update_summary(current_summary, new_info, max_length=1000):
    updated_summary = f'{current_summary} {new_info}'

    def recursive_split(s, delimiters):
        if not delimiters:
            return [s]
        first, *rest = delimiters
        splitted = s.split(first)
        return [sub for segment in splitted for sub in recursive_split(segment, rest)]

    segments = recursive_split(updated_summary, ["\n\n", "\n", " "])
    trimmed_summary = ' '.join(segments[-max_length:])
    reprocessed_summary = process_message(trimmed_summary)
    return reprocessed_summary

def analyze_transcript(state):
    last_message = state['messages'][-1] if state['messages'] else ''
    processed_message = process_message(last_message)  # This is a placeholder for your processing logic
    new_summary = update_summary(state['summary'], processed_message)  # This is a placeholder for your summary logic
    return {'messages': state['messages'], 'summary': new_summary}

def check_keywords_and_update(state):
    keywords = ["hackathon", "drive"]
    latest_transcript = state['messages'][-1].content if state['messages'] else ''

    if 'extracted_info' not in state or state['extracted_info'] is None:
        state['extracted_info'] = {"order": "", "CAPA": False}

    for keyword in keywords:
        found_index = latest_transcript.find(keyword)
        if found_index != -1:
            if keyword == "hackathon":
                # Extract the info related to hackathon for downstream tasks
                start = found_index
                end = min(len(latest_transcript), found_index + 15)
                state['extracted_info'][keyword] = latest_transcript[start:end].strip()
            elif keyword == "drive":
                # Set the flag to True if keyword is mentioned (e.g. drive)
                state['extracted_info'][keyword] = True

    return state


# Initialize your graph
graph = StateGraph(AgentState)

# Add your nodes
graph.add_node("analyze_transcript", analyze_transcript)
graph.add_node("check_keywords_and_update", check_keywords_and_update)

# Define your edges
graph.add_edge("analyze_transcript", "check_keywords_and_update")

# Set entry and exit points
graph.set_entry_point("analyze_transcript")
graph.set_finish_point("check_keywords_and_update")

# Compile the graph
app = graph.compile()

# Sample transcript
# Sample transcript with standard apostrophes
transcript = """
Sarah: "Okay, so we've decided to drive together to San Francisco for the hackathon, right?"
Christos: "Yes, that's settled. I think it's the best choice considering our needs."

Sarah: "Good. Now, about Harrison's role. We've confirmed he's technical enough for our team?"
Christos: "Absolutely. He writes code and handles all the technical demos. He'll be a great asset."

Sarah: "Perfect. That's a relief. Now, regarding the San Jose hackathon, we're going to drive there separately and meet early, correct?"
Christos: "Right. It works out better for our schedules. I'll text you the details of where and when to meet in San Jose."

Sarah: "Sounds good. Lastly, tell me more about the app you've decided to build. I heard it's something special."
Christos: "Yes, it's quite exciting. I'm building the app you're reviewing right now. It integrates advanced features and aims to provide a unique user experience."

Sarah: "Wow, that's impressive! Building the app we're using for this conversation? That's quite meta. I can't wait to see it in action during the hackathons."
Christos: "Thanks! I think it will demonstrate a great blend of creativity and technical skill. Looking forward to collaborating on this."
Sarah: "Me too. Let's catch up again tomorrow to finalize everything."
Christos: "Agreed. See you then!"
"""

# Process each line as a separate message using streaming
transcript_lines = transcript.split('\n')
for line in transcript_lines:
    input_message = {"messages": [HumanMessage(content=line)]}
    for output in app.stream(input_message):
        # 'stream' yields dictionaries with output keyed by node name
        for key, value in output.items():
            print(f"Output from node '{key}':")
            print("---")
            print(value)
        print("\n---\n")

Output from node 'analyze_transcript':
---
{'messages': [HumanMessage(content='')], 'summary': 'The message is blank and does not have any information.'}

---

Output from node 'check_keywords_and_update':
---
{'messages': [HumanMessage(content=''), HumanMessage(content='')], 'summary': 'The message is blank and does not have any information.', 'extracted_info': {'order': '', 'CAPA': False}}

---

Output from node '__end__':
---
{'messages': [HumanMessage(content=''), HumanMessage(content=''), HumanMessage(content=''), HumanMessage(content='')], 'summary': 'The message is blank and does not have any information.', 'extracted_info': {'order': '', 'CAPA': False}}

---

Output from node 'analyze_transcript':
---
{'messages': [HumanMessage(content='Sarah: "Okay, so we\'ve decided to drive together to San Francisco for the hackathon, right?"')], 'summary': 'Sarah is proposing the idea of carpooling to San Francisco for the hackathon.'}

---

Output from node 'check_keywords_and_update':
---

Todd's Thought: Event Driven Website (data with different colors like the keyword-extraction buttons)

In [6]:
agenda = """
Sarah and Christos are meeting to discuss their involvement in two hackathons in the next two weekends.

- How to get to SF on Fri
- Is Harrison technical?
- How to get to SJ on Fri and Sun next week
- What is Christos going to build?
- What functionality to add to the hackathon app next?
"""

def todo_from_agenda():
    prompt = f"Create a todo list from the following agenda:\n{agenda}\n\nOutput should be a python list and nothing else."
    response = model.invoke(prompt)
    return eval(response.content)

todo = todo_from_agenda()

def done(item, completed_todo):
    # Add the resolved item to the completed_todo list
    completed_todo.append(item)

def got_resolved(item, transcript):
    resolution_eval_prompt = f"Instructions:\n Did {item} get resolved in meeting? Transcript: {transcript}\n Output either True or False. Output:"
    resolution_response = model.invoke(resolution_eval_prompt)
    return resolution_response.content.strip()

completed_todo = []

for item in todo:
    # Check if the item got resolved
    if got_resolved(item, transcript):
        done(item, completed_todo)

completed_todo

['How to get to SF on Fri',
 'Is Harrison technical?',
 'How to get to SJ on Fri and Sun next week',
 'What is Christos going to build?',
 'What functionality to add to the hackathon app next?']

In [7]:
def generate_meeting_review(agenda, transcript, todo, completed_todo):
    report = "Meeting Review\n\n"

    # Adding Summary
    report += "Meeting Summary:\n" + "This meeting focused on discussing the logistics and technical aspects of the upcoming hackathons. Key decisions were made regarding travel arrangements and team roles.\n\n"

    # Action items (assuming no specific attendees are mentioned)
    report += "Action Items:\n"

    # Marking items as done or unresolved
    for item in todo:
        status = " ✅" if item in completed_todo else " ❌"
        report += f"- {item}{status}\n"

    return report

# Generate the meeting review
meeting_review = generate_meeting_review(agenda, transcript, todo, completed_todo)

print(meeting_review)

Meeting Review

Meeting Summary:
This meeting focused on discussing the logistics and technical aspects of the upcoming hackathons. Key decisions were made regarding travel arrangements and team roles.

Action Items:
- How to get to SF on Fri ✅
- Is Harrison technical? ✅
- How to get to SJ on Fri and Sun next week ✅
- What is Christos going to build? ✅
- What functionality to add to the hackathon app next? ✅



In [8]:
def extract_resolution_details(item, transcript, model):
    lines = transcript.split("\n")
    for line in lines:
        some_extraction_prompt = f"Check if the following line from a meeting transcript mentions and resolves the agenda item '{item}'. If yes, extract the resolution. If no, return 'None'.\n\nLine: '{line}'\nAgenda Item: '{item}'\n\nResolution:"

        resolution_response = model.invoke(some_extraction_prompt)
        response_content = resolution_response.content.strip()

        if response_content and response_content.lower() != 'none':
            return response_content
    return None

def generate_meeting_review(agenda, transcript, todo, completed_todo, model):
    report = "Meeting Review\n\n"

    # Adding Summary
    report += "Meeting Summary:\nThis meeting focused on discussing the logistics and technical aspects of the upcoming hackathons. Key decisions were made regarding travel arrangements and team roles.\n\n"

    # Action items
    report += "Action Items:\n"

    # Marking items as done or unresolved and adding resolution details
    for item in todo:
        resolution_detail = extract_resolution_details(item, transcript, model)
        if resolution_detail:
            status = f" ✅ ({resolution_detail})"
        else:
            status = " ❌"
        report += f"- {item}{status}\n"

    return report

# Generate the meeting review
meeting_review = generate_meeting_review(agenda, transcript, todo, completed_todo, model)
print(meeting_review)


Meeting Review

Meeting Summary:
This meeting focused on discussing the logistics and technical aspects of the upcoming hackathons. Key decisions were made regarding travel arrangements and team roles.

Action Items:
- How to get to SF on Fri ❌
- Is Harrison technical? ✅ (Yes. The line does mention the agenda item 'Is Harrison technical?' and the resolution is that they have confirmed he is technical enough for their team.)
- How to get to SJ on Fri and Sun next week ✅ (Yes, the line from the meeting transcript mentions and resolves the agenda item 'How to get to SJ on Fri and Sun next week'. The resolution is that Christos will text the details of where and when to meet in San Jose.)
- What is Christos going to build? ✅ (Yes, the line from the meeting transcript mentions and resolves the agenda item 'What is Christos going to build?'. The resolution is "Yes, that's settled. I think it's the best choice considering our needs.")
- What functionality to add to the hackathon app next? ❌



It's not perfect, but neither am I. Just functional at this point. 😀